# semantic segmentation

In [2]:
from ultralytics import YOLO
import yaml
import os
import shutil
from datetime import datetime


def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def load_model(model_path='yolov8n-seg.pt'):
    return YOLO(model_path)


def define_paths(base_path):
    paths = {
        'base': base_path,
        'train_images': os.path.join(base_path, 'images/train'),
        'val_images': os.path.join(base_path, 'images/val'),
        'test_images': os.path.join(base_path, 'images/test'),
        'train_labels': os.path.join(base_path, 'labels/train'),
        'val_labels': os.path.join(base_path, 'labels/val'),
        'test_labels': os.path.join(base_path, 'labels/test')
    }
    return paths


def check_paths(paths):
    for key, path in paths.items():
        if 'test' not in key:  # Test paths are optional
            assert os.path.exists(path), f"{key} path not found: {path}"


def create_dataset_dict(paths, nc, names):
    dataset = {
        'path': paths['base'],
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test' if os.path.exists(paths['test_images']) else '',
        'names': names,
        'nc': nc
    }
    return dataset


def save_dataset_yaml(dataset, filename='temp_data.yaml'):
    with open(filename, 'w') as outfile:
        yaml.dump(dataset, outfile, default_flow_style=False)
    return filename


def train_model(model, data_yaml, save_dir):
    model.train(data=data_yaml, epochs=100, imgsz=640, device='cuda', task='segment', save_dir=save_dir)


def evaluate_model(model, data_yaml, test_images, test_labels):
    if os.path.exists(test_images) and os.path.exists(test_labels):
        results = model.val(data=data_yaml, split='test')
        print("Test evaluation results:", results)


def move_best_model(runs_dir, target_dir):
    best_model_path = os.path.join(runs_dir, 'weights/best.pt')
    target_model_path = os.path.join(target_dir, 'best_model.pt')
    if os.path.exists(best_model_path):
        shutil.move(best_model_path, target_model_path)
        print(f"Best model saved at: {target_model_path}")
    return target_model_path


def main():
    dataset_base_path = '../obj_det_@_tracking/factory_dataset/yolo_dataset/dataset_semantic'
    models_base_dir = '../obj_det_@_tracking/trained_models'
    nc = 2  # number of classes
    names = ["folding_machine", "worker"]  # list of class names

    model = load_model()
    paths = define_paths(dataset_base_path)
    check_paths(paths)

    dataset = create_dataset_dict(paths, nc, names)
    data_yaml = save_dataset_yaml(dataset)

    # Create a new directory for this training run under trained_models
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    model_save_dir = os.path.join(models_base_dir, timestamp)
    ensure_dir(model_save_dir)

    try:
        # Train the model and save results directly in the timestamped folder under trained_models
        model.train(data=data_yaml, epochs=100, imgsz=640, device='cuda', task='segment', save_dir=model_save_dir)

        # Evaluate the model on the test dataset
        evaluate_model(model, data_yaml, paths['test_images'], paths['test_labels'])

        # Move the best model directly to the model_save_dir (not inside runs)
        move_best_model(model_save_dir, model_save_dir)
    finally:
        # Clean up by removing the temporary dataset YAML file
        if os.path.exists(data_yaml):
            os.remove(data_yaml)


if __name__ == '__main__':
    main()


New https://pypi.org/project/ultralytics/8.2.57 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48  Python-3.11.0 torch-2.3.1+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: -1
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
